In [1]:
import sys
sys.path.append('../')
sys.path.append("../input/tez-lib/")
sys.path.append("../input/timmmaster/")
sys.path.append("../input/matsuda-utils")

In [2]:
import numpy as np, pandas as pd
from glob import glob
import shutil, os
import pickle
import matplotlib.pyplot as plt
from sklearn.model_selection import GroupKFold
from sklearn.decomposition import IncrementalPCA
from tqdm.notebook import tqdm
import torch.nn.functional as F
from albumentations.pytorch.transforms import ToTensorV2
import seaborn as sns
import PIL.Image as Image
import torch
from torch.utils.data import Dataset, DataLoader
import cv2
from cuml.svm import SVR
import time
import pandas_profiling as pdp
from pathlib import Path
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold
from utils.util import *
from utils.losses import *
import torch.nn as nn
import math
import transformers as T
import albumentations
import pandas as pd
import cv2
import numpy as np
import timm
import torch.nn as nn
from sklearn import metrics
import torch
from tqdm import tqdm
import math
import albumentations
import tez
import torch.optim as optim
import warnings
warnings.simplefilter('ignore')

In [3]:
class CONFIG:
    DATA_PATH = Path('../input/petfinder-pawpularity-score')
    MODEL_NAME = 'swin_large_patch4_window12_384'
    MODEL_PATH = Path(f'../input/pretrained_models/swin_large_patch4_window12_384_add_petcategory/')
    batch_size = 32
    image_size = 384
    fold = 5
    device='cuda'

In [4]:
test_df = pd.read_csv(CONFIG.DATA_PATH / 'test.csv')
test_df['path'] = test_df['Id'].map(lambda x:str(CONFIG.DATA_PATH/'test'/x)+'.jpg')
test_df['image_size'] = test_df['path'].apply(lambda image_id : Image.open(image_id).size)
test_df['width'] = test_df['image_size'].apply(lambda x: x[0])
test_df['height'] = test_df['image_size'].apply(lambda x: x[1])

In [5]:
test_aug = albumentations.Compose(
    [
        albumentations.Resize(CONFIG.image_size, CONFIG.image_size, p=1),
        albumentations.Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225],
            max_pixel_value=255.0,
            p=1.0,
        ),
    ],
    p=1.0,
)

In [6]:
class PawpularDataset:
    def __init__(self, df, dense_features, targets, augmentations):
        self.image_paths = df['path'].tolist()
        self.dense_features = dense_features
        self.targets = targets
        if self.targets is None:
            self.targets = torch.ones(len(self.image_paths))
        self.augmentations = augmentations
        
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, item):
        image = cv2.imread(self.image_paths[item])
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        if self.augmentations is not None:
            augmented = self.augmentations(image=image)
            image = augmented["image"]
        targets = self.targets[item]
        image = np.transpose(image, (2, 0, 1)).astype(np.float32)
        
        features = self.dense_features[item, :]
        
        return {
            "image": torch.tensor(image, dtype=torch.float),
            "features": torch.tensor(features, dtype=torch.float),
            "targets": torch.tensor(targets, dtype=torch.float)
        }

In [7]:
class PawpularModel(nn.Module):
    def __init__(self, pet_classify_model, model_name):
        super().__init__()
        self.pet_classify_model = pet_classify_model
        self.pet_classify_model.requires_grad = False
        self.model = timm.create_model(model_name, pretrained=False, in_chans=3)
        self.model.head = nn.Linear(self.model.head.in_features, 128)
        self.dropout = nn.Dropout(0.1)
        self.dense1 = nn.Linear(177, 64)
        self.dense2 = nn.Linear(64, 1)

    def forward(self, image, features):
        p = self.pet_classify_model(F.adaptive_avg_pool2d(image, (224,224)))
        p = torch.softmax(p, dim=1)
        x1 = self.model(image)
        x = self.dropout(x1)
        x = torch.cat([x, features, p], dim=1)
        x = self.dense1(x)
        x = self.dense2(x)
        return torch.cat([x, x1, features, p], dim=1)
    
class pet_categor_extract_model(nn.Module):
    def __init__(self,class_num):
        super().__init__()
        self.model = timm.create_model('efficientnet_b0', pretrained=True, in_chans=3)
        self.model.classifier = nn.Linear(self.model.classifier.in_features, 128)
        self.dropout = nn.Dropout(0.1)
        self.dense = nn.Linear(128
                               , class_num)

    def forward(self, image):
        x = self.model(image)
        x = self.dropout(x)
        x = self.dense(x)
        return x.squeeze(1)

In [8]:
def extract_feature(model, df):
    
    
    model.eval()
    
    dense_features = [
        'Subject Focus', 'Eyes', 'Face', 'Near', 'Action', 'Accessory',
        'Group', 'Collage', 'Human', 'Occlusion', 'Info', 'Blur'
    ]
    df_folds = df.reset_index(drop=True)
    
    dataset = PawpularDataset(
        df_folds, 
        dense_features=df_folds[dense_features].values, targets=None,
        augmentations=test_aug
    )
    
    loader = DataLoader(
        dataset,
        batch_size=CONFIG.batch_size,
        shuffle=False,
        pin_memory=True,
        num_workers=4,
        drop_last=False,
    )

    embedding_feature = []
    
    for iter, data in tqdm(enumerate(loader), total=len(loader)):
        img, feature ,target = data['image'],  data['features'], data['targets']
        img = img.to(CONFIG.device)
        feature = feature.to(CONFIG.device)
        target = target.to(CONFIG.device)
        batch_size = target.size(0)
        with torch.no_grad():
            embedding = model(img, feature)
        embedding_feature.append(embedding.data.cpu())
    
    embedding_feature = np.concatenate(embedding_feature)

    return embedding_feature[:,:1].ravel().tolist(), embedding_feature[:,1:]

In [9]:
def sigmoid(x):
    return 1 / (1 + math.exp(-x))
def main():
    super_final_predictions = []
    super_final_predictions2 = []
    super_final_oof_predictions = []
    super_final_oof_predictions2 = []
    super_final_oof_true = []
    for fold in range(CONFIG.fold):

        # ====================================================
        # Data Loader
        # ====================================================
        cl_model = pet_categor_extract_model(class_num=37)
        cl_model.to(CONFIG.device)
        model = PawpularModel(cl_model, model_name=CONFIG.MODEL_NAME)
        model.to(CONFIG.device)

        if torch.cuda.device_count()>1:
            model=nn.DataParallel(model)
            model.load_state_dict(torch.load(CONFIG.MODEL_PATH / f"{CONFIG.MODEL_NAME}_{fold}_best.pth")["model"])
        else:
            model.load_state_dict(fix_model_state_dict(torch.load(CONFIG.MODEL_PATH / f"{CONFIG.MODEL_NAME}_{fold}_best.pth")["model"]))
            
        test_folds = test_df.reset_index(drop=True)
        preds_test, embed_test = extract_feature(model, test_folds)
        print('Loading SVR...')
        clf = pickle.load(open(CONFIG.MODEL_PATH / f"SVR_fold_{fold}.pkl", "rb"))
        ##fit SVR to test data
        test_pred_SVR = clf.predict(embed_test.astype('float32'))
        test_pred_NN  =[sigmoid(x) * 100 for x in preds_test]
        
        super_final_predictions.append(test_pred_SVR)
        super_final_predictions2.append(test_pred_NN)
        
    # submission
    submission = test_df.copy()
    
    best_w = 0.5
    super_final_predictions = np.mean(np.column_stack(super_final_predictions), axis=1)
    super_final_predictions2 = np.mean(np.column_stack(super_final_predictions2), axis=1)
    submission["Pawpularity"] = (1-best_w)*super_final_predictions + best_w*super_final_predictions2
    submission = submission[["Id", "Pawpularity"]]
    submission.to_csv("submission.csv", index=False)

In [10]:
if __name__ == "__main__":
    main()

100%|██████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.97s/it]


Loading SVR...


100%|██████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  2.87it/s]


Loading SVR...


100%|██████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  2.95it/s]


Loading SVR...


100%|██████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  2.73it/s]


Loading SVR...


100%|██████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  3.28it/s]

Loading SVR...
